In [1]:
# setting the environment variables
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

from config import set_environment
set_environment()

In [2]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Choose a more up-to-date model
checkpoint = "google/codegemma-2b"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Create a text generation pipeline
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500
)

# Integrate the pipeline with LangChain
llm = HuggingFacePipeline(pipeline=pipe)

# Define the input text
text = """
def calculate_primes(n):
    \"\"\"Create a list of consecutive integers from 2 up to N.

    For example:
    >>> calculate_primes(20)
    Output: [2, 3, 5, 7, 11, 13, 17, 19]
    \"\"\"
"""

# Use the LangChain LLM to generate text
output = llm(text)
print(output)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
/tmp/ipykernel_7940/3744229729.py:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
/tmp/ipykernel_7940/3744229729.py:34: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = llm(text)



def calculate_primes(n):
    """Create a list of consecutive integers from 2 up to N.

    For example:
    >>> calculate_primes(20)
    Output: [2, 3, 5, 7, 11, 13, 17, 19]
    """
    primes = []
    for i in range(2, n + 1):
        if is_prime(i):
            primes.append(i)
    return primes

def is_prime(n):
    """Return True if n is prime."""
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

def main():
    """Get user input and print the list of primes."""
    n = int(input("Enter a number: "))
    primes = calculate_primes(n)
    print(primes)

if __name__ == "__main__":
    main()
<|file_separator|>


Please note that downloading and loading of the models can take a few minutes. If you're getting an error saying you "cannot access a gated repo" when trying to use a URL with Langchain, it means you're attempting to access a private repository on Hugging Face that requires authentication with a personal access token to view or use the model; you need to create a Hugging Face access token and set it as an environment variable named "HF_TOKEN" to access the gated repository. 

In [3]:
completion = pipe(text)
print(completion[0]["generated_text"])


def calculate_primes(n):
    """Create a list of consecutive integers from 2 up to N.

    For example:
    >>> calculate_primes(20)
    Output: [2, 3, 5, 7, 11, 13, 17, 19]
    """
    primes = []
    for i in range(2, n + 1):
        if is_prime(i):
            primes.append(i)
    return primes

def is_prime(n):
    """Return True if n is prime."""
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

def main():
    """Get user input and print the list of primes."""
    n = int(input("Enter a number: "))
    primes = calculate_primes(n)
    print(primes)

if __name__ == "__main__":
    main()
<|file_separator|>


In [48]:
from huggingface_hub import InferenceClient
from langchain_core.prompts import PromptTemplate

client = InferenceClient()
WORKING_MODEL = "HuggingFaceH4/zephyr-7b-beta"

def simple_code_completion(code):
    response = client.chat.completions.create(
        model=WORKING_MODEL,
        messages=[{"role": "user", "content": f"Continue this code: {code}\nAnswer: Let's think step by step."}],
        max_tokens=128,
        temperature=0.5
    )
    return response.choices[0].message.content

# Test it
result = simple_code_completion(text)
print(result)

1. First, we'll create an empty list to store our primes.

```python
primes = []
```

2. Next, we'll create a list of consecutive integers from 2 up to N using a list comprehension.

```python
numbers = [i for I in range(2, n+1)]
```

3. We'll iterate over our list of numbers and mark multiples of each prime as composite.

```python
for p in primes:
    numbers[i:i+p


In [49]:
from langchain_anthropic import ChatAnthropic

from langchain_core.prompts.prompt import PromptTemplate

template = """Question: {question}
Let's think step by step.

Answer:
"""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm = ChatAnthropic(model='claude-3-opus-20240229')

llm_chain = prompt | llm
print(llm_chain.invoke(text))

content="Sure! Let's break down the problem and solve it step by step.\n\nTo create a list of prime numbers up to a given number N, we can follow these steps:\n\n1. Create a list of consecutive integers from 2 to N.\n2. Iterate through the list and mark the composite numbers (non-prime numbers) as 0.\n3. Remove the marked composite numbers from the list to get the prime numbers.\n\nHere's the Python code that implements these steps:\n\n```python\ndef calculate_primes(n):\n    # Create a list of consecutive integers from 2 to n\n    numbers = list(range(2, n + 1))\n\n    # Iterate through the list\n    for i in range(2, int(n ** 0.5) + 1):\n        if numbers[i - 2] != 0:\n            # Mark the composite numbers as 0\n            for j in range(i * 2, n + 1, i):\n                numbers[j - 2] = 0\n\n    # Remove the marked composite numbers from the list\n    primes = [num for num in numbers if num != 0]\n\n    return primes\n```\n\nLet's go through each step of the code:\n\n1. We cre

In [50]:
from langchain_core.language_models.fake import FakeListLLM
from langchain.agents import initialize_agent, AgentType
from langchain_experimental.tools import PythonREPLTool

responses = ["Action: Python_REPL\nAction Input: print(2 + 2)", "Final Answer: 4"]
llm = FakeListLLM(responses=responses)
tools = [PythonREPLTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
result = agent("What is 2 + 2?")
print(result)

/tmp/ipykernel_7940/4035251697.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
/tmp/ipykernel_7940/4035251697.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent("What is 2 + 2?")
Pytho



> Entering new AgentExecutor chain...
Action: Python_REPL
Action Input: print(2 + 2)
Observation: 4

Thought:Final Answer: 4

> Finished chain.
{'input': 'What is 2 + 2?', 'output': '4'}


In [51]:
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

llm = ChatOpenAI()
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
result = agent("What are the prime numbers until 20?")
print(result)



> Entering new AgentExecutor chain...
I can write a function to check if a number is prime and then loop through numbers 2 to 20 to find prime numbers.
Action: Python_REPL
Action Input: 
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5)+1):
        if n % i == 0:
            return False
    return True

primes = [num for num in range(2, 21) if is_prime(num)]
print(primes)
Observation: [2, 3, 5, 7, 11, 13, 17, 19]

Thought:The prime numbers until 20 are 2, 3, 5, 7, 11, 13, 17, 19.
Final Answer: 2, 3, 5, 7, 11, 13, 17, 19

> Finished chain.
{'input': 'What are the prime numbers until 20?', 'output': '2, 3, 5, 7, 11, 13, 17, 19'}
